In [1]:
import pandas as pd
import json

In [2]:
with open('../data/data_salto/concursos/concursos_salto_nacional_2025.json', 'r', encoding='utf-8') as file:
    datos_concursos = json.load(file)

df_concursos = pd.DataFrame(datos_concursos)
df_concursos = df_concursos.rename(columns = {"Nombre": "Concurso", "Categoría": "Categoria_concurso"})
df_concursos["Concurso"] = df_concursos["Concurso"].str.strip()
df_concursos.head()

,Concurso,Categoria_concurso,Provincia,Localidad,Disciplina,Federación,Resultados,País,Inicio,Final,Ambito
0,CSN3* Invierno,CSN***,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,10/01/2025,12/01/2025,Nacional
1,SPRING MET I 2025 YH*,CSI YH*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,14/01/2025,16/01/2025,Internacional
2,SPRING MET I 2025 - SILVER 1*,CSI*,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,18/01/2025,Internacional
3,SPRING MET I 2025 - GOLD 2*,CSI**,Valencia,Centro Ecuestre Oliva Nova,Salto de Obstáculos,Federación Hípica Comunidad Valenciana,Ver,Epaña,16/01/2025,19/01/2025,Internacional
4,CSIYH* - 1st Week Andalucía Pre-Sunshine Tour ...,CSI YH*,Cádiz,Dehesa Montenmedio,Salto de Obstáculos,Federación Hípica Andaluza,Ver,Epaña,16/01/2025,19/01/2025,Internacional


In [3]:
df_pruebas_resultados = pd.read_csv("../data/data_salto/resultados_combinados_2025.csv", index_col=0)
df_pruebas_resultados.head()

C:\Users\gabri\AppData\Local\Temp\ipykernel_15888\1077302799.py:1: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pruebas_resultados = pd.read_csv("../data/data_salto/resultados_combinados_2025.csv", index_col=0)


,Estado,Posicion,Lic_jinete,Jinete,Pais_jinete,Lic_caballo,Caballo,Raza_caballo,Pais_caballo,Puntuacion,...,Categoria,Ptos_obs_R1,Ptos_tiempo_R1,Tiempo_R1,Ptos_obs_R2,Ptos_tiempo_R2,Tiempo_R2,Ptos_obs_R3,Ptos_tiempo_R3,Tiempo_R3
0,FIN,1,284725,Pedro Miguel DA SILVA MANSO,ESP,169882,BRIDGETOWN JONES OS,(Indt.),ESP,"0/59,91",...,CSN***,0.0,NaN,59.91,NaN,NaN,NaN,NaN,NaN,NaN
1,FIN,2,77612,Ignacio ASTOLFI LÓPEZ,ESP,157373,QUINTO BLUE PS,OLDBG,ESP,"0/63,23",...,CSN***,0.0,NaN,63.23,NaN,NaN,NaN,NaN,NaN,NaN
2,FIN,3,306183,Francisco PICAO CALDEIRA,ESP,145996,FLOR DU PASO,CRPOR,ESP,"0/65,58",...,CSN***,0.0,NaN,65.58,NaN,NaN,NaN,NaN,NaN,NaN
3,FIN,4,1004,Jesús TORRES GARCÍA,ESP,170075,DIABETTO PS,OLDBG,ESP,"0/66,00",...,CSN***,0.0,NaN,66.00,NaN,NaN,NaN,NaN,NaN,NaN
4,FIN,5,367363,Nina FAGERSTROM,ESP,170155,HASENACHER CERVIN,ZANG,ESP,"0/66,78",...,CSN***,0.0,NaN,66.78,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
& (df_final["Categoria"] == df_final["Categoria_concurso"])

In [4]:
df_final = pd.merge(df_pruebas_resultados, df_concursos, on = "Concurso", how = "inner")
df_final['Fecha_prueba'] = pd.to_datetime(df_final['Fecha_prueba'])
df_final['Inicio'] = pd.to_datetime(df_final['Inicio'], dayfirst=True)
df_final['Final'] = pd.to_datetime(df_final['Final'], dayfirst=True)
df_final = df_final[(df_final['Fecha_prueba'] >= df_final['Inicio']) & (df_final['Fecha_prueba'] <= df_final['Final'])]
df_final = df_final.drop(['Puntuacion', 'Resultados', 'Categoria_concurso', 'Pais_jinete', 'Pais_caballo', 'Raza_caballo'], axis=1)
df_final["Jinete"] = df_final["Jinete"].str.lower()
df_final["Caballo"] = df_final["Caballo"].str.lower()
df_final["Lic_jinete"] = df_final["Lic_jinete"].astype(str)
df_final["Lic_caballo"] = df_final["Lic_caballo"].astype(str)
# df_final["Raza_caballo"] = df_final["Raza_caballo"].apply(lambda x: None if x == '(Indt.)' else x)
df_final["Lic_jinete"] = df_final["Lic_jinete"].apply(lambda x: "10186822" if x == '--' else x)
df_final["Jinete"] = df_final["Jinete"].apply(lambda x: "jesus folch redondo" if x == '-- -- --' else x)
lista = ["ELI", "RET", "NOP"]
df_final["Posicion"] = df_final["Posicion"].apply(lambda x: None if x in lista else x)
df_final["Lic_caballo"] = df_final["Lic_caballo"].apply(lambda x: None if x == '--' else x)

lic_mas_larga_por_jinete = (
    df_final[["Jinete", "Lic_jinete"]]
    .drop_duplicates()
    .assign(longitud=lambda x: x["Lic_jinete"].astype(str).str.len())
    .sort_values("longitud", ascending=False)
    .drop_duplicates(subset="Jinete")
    .drop(columns="longitud")
    .set_index("Jinete")
)

lic_mas_larga_por_caballo = (
    df_final[["Caballo", "Lic_caballo"]]
    .drop_duplicates()
    .assign(longitud=lambda x: x["Lic_caballo"].astype(str).str.len())
    .sort_values("longitud", ascending=False)
    .drop_duplicates(subset="Caballo")
    .drop(columns="longitud")
    .set_index("Caballo")
)

# 2. Reemplazar en el DataFrame original
df_final["Lic_jinete"] = df_final["Jinete"].map(lic_mas_larga_por_jinete["Lic_jinete"])
df_final["Lic_caballo"] = df_final["Caballo"].map(lic_mas_larga_por_caballo["Lic_caballo"])
df_final.head()


,Estado,Posicion,Lic_jinete,Jinete,Lic_caballo,Caballo,Premio,Dinero_premio,Prueba,Fecha_prueba,...,Ptos_tiempo_R3,Tiempo_R3,Provincia,Localidad,Disciplina,Federación,País,Inicio,Final,Ambito
0,FIN,1,284725,pedro miguel da silva manso,169882,bridgetown jones os,True,450.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
1,FIN,2,77612,ignacio astolfi lópez,157373,quinto blue ps,True,300.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
2,FIN,3,306183,francisco picao caldeira,145996,flor du paso,True,200.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
3,FIN,4,10006303,jesús torres garcía,107ZP87,diabetto ps,True,150.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional
4,FIN,5,10002400,nina fagerstrom,105ZA09,hasenacher cervin,True,125.0,"1,30 - A c.c. (238.2.1)",2025-01-10,...,NaN,NaN,Sevilla,Real Club Pineda,Salto de Obstáculos,Federación Hípica Andaluza,Epaña,2025-01-10,2025-01-12,Nacional


In [5]:
df_final.to_csv("../data/data_salto/informacion_concursos_final.csv")

In [6]:
df = pd.read_csv("../data/data_salto/informacion_concursos_final.csv", index_col = 0)
df["Disciplina"] = "Salto de Obstaculos"
i = 0
df["id_resultado"] = range(i +1 , i + 1 + len(df))
print(df["Lic_jinete"].duplicated().sum())
print(df["Jinete"].duplicated().sum())

69458
69458


C:\Users\gabri\AppData\Local\Temp\ipykernel_15888\1380352046.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/data_salto/informacion_concursos_final.csv", index_col = 0)


In [8]:
df_coso = df[["Concurso", "Prueba"]]
concurso_pruebas = {}
concurso_pruebas = {}
for _, row in df_coso.iterrows():  # Usar el dataframe df que proporcionaste
    concurso = row["Concurso"]
    prueba = row["Prueba"]
    if concurso in concurso_pruebas:
        if prueba not in concurso_pruebas[concurso]: # Añadido esta condición
            concurso_pruebas[concurso].append(prueba)
    else:
        concurso_pruebas[concurso] = [prueba]

concurso_pruebas

{'CSN3* Invierno': ['1,30 - A c.c. (238.2.1)',
  '1,20 - A c.c. (238.2.1)',
  '1,25 - Dos Fases Especial (274.2)',
  '1,40 - A c.c. (238.2.1)',
  '1.35 GP - Acc y desempate (238.2.2)',
  '1,30 - Dos fases Especial (Art. 274.2) (274.2)',
  '1,40 gp - A.c.c. y desempate al cronómetro (238.2.2)',
  '1,35 - Dos fases Especial (Art. 274.2) (274.2)'],
 'SPRING MET I 2025 YH*': ['6 años - 1,25 - A sin cronómetro (238.1.1)',
  '7 años - 1,35 - A sin cronómetro (238.1.1)',
  '5 años - 1,15 - A.c.c. y desempate (238.2.2)',
  '6 años - 1,25 - A.c.c. y desempate (238.2.2)',
  '7 años - 1,35 - A.c.c. y desempate (238.2.2)',
  '5 años - 1,15 - Dos Fases Especial (274.2)',
  '6 años - 1,25 - Dos Fases Especial (274.2)',
  '7 años - 1,35 - Dos Fases Especial (274.2)',
  '5 años - 1,15 - A sin cronómetro (238.1.1)'],
 'MOURA TOURS VALENCIA - SPRING TOUR': ['1,35 (G) - A c.c. (238.2.1)',
  '1,30 - A c.c. (238.2.1)',
  '1,40 (G) - DOS FASES (A c/c - A c/c) (274.1.5.3)',
  '1,45 (G) - A c.c. (238.2.1)',
 

In [ ]:
import psycopg2 as ps

In [ ]:
conn = ps.connect(
    dbname = "BBDD_Hipica", # base a la que nos queremos conectar
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432" # puerto en el que s eencuentra postgres
)

In [ ]:
cur = conn.cursor()

In [ ]:
# COmprobamos que la conexión está creada y conectada
cur.execute("SELECT version();")
cur.fetchone() 

In [ ]:
data_insert_disciplinas = ["Salto de Obstaculos", "Concurso Completo", "Doma clasica"]
# Creamos la query de inserción de los datos de ciudad
insert_query_disciplinas = """ 
                        INSERT INTO disciplinas (disciplina)
                        VALUES (%s) 
"""
data_to_insert = [(disciplina,) for disciplina in data_insert_disciplinas]
# Ejecutamos la query para insertar las disciplinas
cur.executemany(insert_query_disciplinas, data_to_insert)
conn.commit()
df_caballos = df[["Lic_caballo", "Caballo"]]
# Creamos la lista que va a contener toda la información 
data_insert_caballos = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_caballos.iterrows():
    id_caballo = row["Lic_caballo"]
    nombre_caballo = row["Caballo"]

    tupla_caballo = (id_caballo, nombre_caballo)

    if tupla_caballo not in data_insert_caballos:
        data_insert_caballos.append(tupla_caballo)
insert_query_caballos = """ 
                        INSERT INTO caballos (id_caballo, nombre_caballo)
                        VALUES (%s, %s)
"""    
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_caballos, data_insert_caballos)
conn.commit()  
df_jinetes = df[["Lic_jinete", "Jinete"]]
# Creamos la lista que va a contener toda la información 
data_insert_jinetes = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_jinetes.iterrows():
    id_jinete = row["Lic_jinete"]
    nombre_jinete = row["Jinete"]

    tupla_jinete = (id_jinete, nombre_jinete)

    if tupla_jinete not in data_insert_jinetes:
        data_insert_jinetes.append(tupla_jinete)
insert_query_jinetes = """ 
                        INSERT INTO jinetes (id_jinete, nombre_jinete)
                        VALUES (%s, %s)
"""
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_jinetes, data_insert_jinetes)
conn.commit()
df_concursos = df[["Concurso", "Categoria", "País", "Provincia", "Localidad", "Ambito", "Federación", "Inicio", "Final"]]
df_concursos
# Creamos la lista que va a contener toda la información 
data_insert_concursos = []

# Vamos iterando por cada columna del dataframe y cada fila de cada columna, obteniendo los elementos de las filas (row)
for i, row in df_concursos.iterrows():
    nombre_concurso = row["Concurso"]
    categoria_concurso = row["Categoria"]
    pais_concurso = row["País"]
    provincia_concurso = row["Provincia"]
    localidad_concurso = row["Localidad"]
    ambito_concurso = row["Ambito"]
    federacion_concurso = row["Federación"]
    inicio_concurso = row["Inicio"]
    final_concurso = row["Final"]

    tupla_concursos = (nombre_concurso, categoria_concurso, pais_concurso, provincia_concurso, 
                       localidad_concurso, ambito_concurso, federacion_concurso, 
                       inicio_concurso, final_concurso)

    if tupla_concursos not in data_insert_concursos:
        data_insert_concursos.append(tupla_concursos)
insert_query_concursos = """ 
                        INSERT INTO concursos (nombre_concurso, categoria_concurso, 
                                               pais_concurso, provincia_concurso, 
                                               localidad_concurso, ambito_concurso,
                                               federacion_concurso, fecha_inicio_concurso,
                                               fecha_fin_concurso)
                        VALUES (%s, %s,  %s, %s, %s, %s, %s, %s, %s)
"""
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_concursos, data_insert_concursos)
conn.commit()

cur.execute("SELECT disciplina, id_disciplina FROM disciplinas")
dictio_disciplinas = dict(cur.fetchall())
df_pruebas = df[["Prueba", "Fecha_prueba", "Disciplina"]]
data_insert_pruebas = []

for _, row in df_pruebas.iterrows():
    nombre_prueba = row["Prueba"]
    fecha_prueba = row["Fecha_prueba"]
    disciplina_prueba = dictio_disciplinas.get(row["Disciplina"])

    tupla_pruebas = (nombre_prueba, fecha_prueba, disciplina_prueba)

    if tupla_pruebas not in data_insert_pruebas:
        data_insert_pruebas.append(tupla_pruebas)
insert_query_pruebas = """ 
                        INSERT INTO pruebas (nombre_prueba, fecha_prueba,id_disciplina)
                        VALUES (%s, %s,  %s)
"""
# Ejecutamos la query y le indicamos los valores 
cur.executemany(insert_query_pruebas, data_insert_pruebas)
conn.commit()
cur.execute("SELECT nombre_jinete, id_jinete FROM jinetes")
dictio_jinetes = dict(cur.fetchall())
cur.execute("SELECT nombre_caballo, id_caballo FROM caballos")
dictio_caballos = dict(cur.fetchall())
cur.execute("SELECT nombre_concurso, id_concurso FROM concursos")
dictio_concursos = dict(cur.fetchall())
cur.execute("SELECT nombre_prueba, id_prueba FROM pruebas")
dictio_pruebas = dict(cur.fetchall())
df_resultados = df[["id_resultado", "Jinete", "Caballo", "Concurso", "Prueba", "Posicion", "Estado", "Premio", "Dinero_premio"]]
data_insert_resultados = []
lista = ["ELI", "NOP", "RET", "DSQ", "ret", "nop", "dsq", "eli"]
for _, row in df_resultados.iterrows():
    id_resultado = int(row["id_resultado"])
    id_jinete = dictio_jinetes.get(row["Jinete"])
    id_caballo = dictio_caballos.get(row["Caballo"])
    id_concurso = dictio_concursos.get(row["Concurso"])
    id_prueba = dictio_pruebas.get(row["Prueba"])
    # puesto debe ser None si es NaN o está en lista
    puesto = row["Posicion"] if pd.notna(row["Posicion"]) and row["Posicion"] not in lista else None
    estado = row["Estado"]
    premio = row["Premio"]
    dinero_premio = row["Dinero_premio"]

    tupla_resultados = (id_resultado, id_jinete, id_caballo, id_prueba, id_concurso, puesto, estado, premio, dinero_premio)

    if tupla_resultados not in data_insert_resultados:
        data_insert_resultados.append(tupla_resultados)
insert_query_resultados = """ 
                        INSERT INTO resultados (id_resultado, id_jinete, id_caballo, id_prueba, id_concurso, puesto, estado, premio, dinero_premio)
                        VALUES (%s, %s, %s,  %s, %s, %s, %s, %s, %s)
"""
# Ejecutamos la query y le indicamos los valores 

cur.executemany(insert_query_resultados, data_insert_resultados)
conn.commit()

df_resultados_salto = df[["Disciplina","id_resultado", "Ptos_obs_R1", "Ptos_tiempo_R1", "Tiempo_R1", "Ptos_obs_R2", "Ptos_tiempo_R2", "Tiempo_R2", "Ptos_obs_R3", "Ptos_tiempo_R3", "Tiempo_R3"]].copy()
data_insert_resultados_salto = []
cur.execute("SELECT id_resultado FROM resultados")
resultados_existentes = {row[0]: None for row in cur.fetchall()}

for _, row in df_resultados_salto.iterrows():
    id_resultado = row["id_resultado"]
    if id_resultado not in resultados_existentes:
        print(f"Error: El id_resultado {id_resultado} del dataframe no existe en la tabla resultados")
        continue 

    id_disciplina = dictio_disciplinas.get(row["Disciplina"])

    ptos_obs_r1 = row["Ptos_obs_R1"] if pd.notna(row["Ptos_obs_R1"]) else None
    ptos_tmp_r1 = row["Ptos_tiempo_R1"] if pd.notna(row["Ptos_tiempo_R1"]) else None
    tiempo_r1 = row["Tiempo_R1"] if pd.notna(row["Tiempo_R1"]) else None

    ptos_obs_r2 = row["Ptos_obs_R2"] if pd.notna(row["Ptos_obs_R2"]) else None
    ptos_tmp_r2 = row["Ptos_tiempo_R2"] if pd.notna(row["Ptos_tiempo_R2"]) else None
    tiempo_r2 = row["Tiempo_R2"] if pd.notna(row["Tiempo_R2"]) else None

    ptos_obs_r3 = row["Ptos_obs_R3"] if pd.notna(row["Ptos_obs_R3"]) else None
    ptos_tmp_r3 = row["Ptos_tiempo_R3"] if pd.notna(row["Ptos_tiempo_R3"]) else None
    tiempo_r3 = row["Tiempo_R3"] if pd.notna(row["Tiempo_R3"]) else None


    tupla_resultados_salto = (id_resultado, id_disciplina, ptos_obs_r1, ptos_tmp_r1, tiempo_r1, ptos_obs_r2, ptos_tmp_r2, tiempo_r2, ptos_obs_r3, ptos_tmp_r3, tiempo_r3)

    if tupla_resultados_salto not in data_insert_resultados_salto:
        data_insert_resultados_salto.append(tupla_resultados_salto)
insert_query_resultados_salto = """ 
                        INSERT INTO resultados_salto (id_resultado, id_disciplina, puntos_obs_r1, puntos_tmp_r1, tiempo_r1, puntos_obs_r2, puntos_tmp_r2, tiempo_r2, puntos_obs_r3, puntos_tmp_r3, tiempo_r3)
                        VALUES (%s, %s, %s,  %s, %s, %s, %s, %s, %s, %s, %s)
"""
cur.executemany(insert_query_resultados_salto, data_insert_resultados_salto)
conn.commit()